In [3]:
import pandas as pd
import itertools
import math

# Function to convert American odds to decimal odds
def american_to_decimal(odds):
    odds = float(odds)
    if odds < 0:
        return 1 + (100 / -odds)
    else:
        return 1 + (odds / 100)

# Function to convert decimal odds to American odds
def decimal_to_american(decimal_odds):
    decimal_odds = float(decimal_odds)
    if decimal_odds >= 2:
        return (decimal_odds - 1) * 100
    else:
        return -100 / (decimal_odds - 1)

# Function to calculate implied probability from American odds
def american_to_implied_prob(odds):
    odds = float(odds)
    if odds < 0:
        return -odds / (-odds + 100)
    else:
        return 100 / (odds + 100)

# Read the CSV file
df = pd.read_csv('Next30_Odds.csv')

# List to hold bets on favorites with adjusted odds range
favorite_bets = []

# Process each row to identify favorites
for index, row in df.iterrows():
    try:
        odds1 = float(row['odds1'])
        odds2 = float(row['odds2'])
        team1 = row['team1']
        team2 = row['team2']
        event = f"{row['sport']} {row['event_time']}"
    except (ValueError, KeyError, TypeError):
        continue  # Skip rows with invalid data

    # Identify favorite and underdog
    if odds1 < odds2:
        favorite_odds = odds1
        underdog_odds = odds2
        favorite_team = team1
    else:
        favorite_odds = odds2
        underdog_odds = odds1
        favorite_team = team2

    # Adjusted odds range (-2000 to -200)
    if favorite_odds < 0 and -2000 <= favorite_odds <= -200:
        # Calculate implied probabilities
        favorite_implied_prob = american_to_implied_prob(favorite_odds)
        underdog_implied_prob = american_to_implied_prob(underdog_odds)
        # Calculate discrepancy as ratio
        if underdog_implied_prob > 0:
            discrepancy = favorite_implied_prob / underdog_implied_prob
        else:
            discrepancy = float('inf')
        # Store the bet with discrepancy
        favorite_bets.append({
            'event': event,
            'team': favorite_team,
            'odds': favorite_odds,
            'decimal_odds': american_to_decimal(favorite_odds),
            'implied_prob': favorite_implied_prob,
            'discrepancy': discrepancy
        })

# Check the number of favorite bets
print(f"Number of favorite bets: {len(favorite_bets)}")

# Sort favorite bets by discrepancy (descending) and limit to top K bets
K = 50  # Adjust K as needed
favorite_bets = sorted(favorite_bets, key=lambda x: x['discrepancy'], reverse=True)[:K]

# Pre-calculate the logarithms of decimal odds and implied probabilities
for bet in favorite_bets:
    bet['log_decimal_odds'] = math.log(bet['decimal_odds'])
    bet['log_implied_prob'] = math.log(bet['implied_prob'])

# Generate all combinations of bets (parlays)
parlays = []

for r in range(2, 6):  # Consider parlays of size 2 to 5
    combinations = itertools.combinations(favorite_bets, r)
    for combo in combinations:
        # Ensure all events are unique
        event_names = [bet['event'] for bet in combo]
        if len(set(event_names)) < len(event_names):
            continue  # Skip combinations with duplicate events

        # Calculate combined decimal odds and implied probability
        combined_decimal_odds = math.exp(sum(bet['log_decimal_odds'] for bet in combo))
        combined_prob = math.exp(sum(bet['log_implied_prob'] for bet in combo))
        min_discrepancy = min(bet['discrepancy'] for bet in combo)
        events = [bet['event'] for bet in combo]
        teams = [bet['team'] for bet in combo]
        odds_list = [bet['odds'] for bet in combo]
        # Convert combined decimal odds to American odds
        combined_american_odds = decimal_to_american(combined_decimal_odds)
        # Adjust combined odds range
        if 100 <= combined_american_odds <= 300:
            # Calculate expected value
            payout = combined_decimal_odds - 1  # Profit per $1 bet
            expected_value = (combined_prob * payout) - ((1 - combined_prob) * 1)
            parlays.append({
                'parlay': list(zip(events, teams, odds_list)),
                'combined_odds': combined_american_odds,
                'implied_probability': combined_prob,
                'expected_value': expected_value,
                'min_discrepancy': min_discrepancy
            })

# Check if any parlays were generated
if not parlays:
    print("No parlays generated with the current criteria.")
else:
    # Sort parlays from most likely to least likely based on implied probability and minimum discrepancy
    sorted_parlays = sorted(parlays, key=lambda x: (-x['implied_probability'], -x['min_discrepancy']))

    # Output the results to a file
    with open('parlay_results.txt', 'w') as f:
        for parlay in sorted_parlays:
            f.write("Parlay:\n")
            for event, team, odds in parlay['parlay']:
                f.write(f"  Event: {event}, Team: {team}, Odds: {odds}\n")
            f.write(f"Combined Odds (American): +{parlay['combined_odds']:.2f}\n")
            f.write(f"Implied Probability: {parlay['implied_probability'] * 100:.2f}%\n")
            f.write(f"Minimum Discrepancy: {parlay['min_discrepancy']:.2f}\n")
            f.write(f"Expected Value per $1 bet: ${parlay['expected_value']:.2f}\n")
            f.write("-----\n")

    print("Parlay results have been written to 'parlay_results.txt'.")


Number of favorite bets: 148
Parlay results have been written to 'parlay_results.txt'.


In [1]:
import pandas as pd
import itertools
import math

# Function to convert American odds to decimal odds
def american_to_decimal(odds):
    odds = float(odds)
    if odds < 0:
        return 1 + (100 / -odds)
    else:
        return 1 + (odds / 100)

# Function to convert decimal odds to American odds
def decimal_to_american(decimal_odds):
    decimal_odds = float(decimal_odds)
    if decimal_odds >= 2:
        return (decimal_odds - 1) * 100
    else:
        return -100 / (decimal_odds - 1)

# Function to calculate implied probability from American odds
def american_to_implied_prob(odds):
    odds = float(odds)
    if odds < 0:
        return -odds / (-odds + 100)
    else:
        return 100 / (odds + 100)

# Adjust implied probabilities based on sport rating
def adjust_implied_prob(prob, rating, is_favorite=True):
    adjustment_factor = rating / 5.0  # Normalize rating (5.0 is max)
    if is_favorite:
        # Push favorite probabilities closer to 1
        return prob + (1 - prob) * (adjustment_factor - 1)
    else:
        # Push underdog probabilities closer to 0
        return prob * (1 - adjustment_factor)

# Function to generate parlays
def generate_parlays(favorite_bets, min_parlay_size, max_parlay_size, odds_lower, odds_upper):
    parlays = []
    for r in range(min_parlay_size, max_parlay_size + 1):
        combinations = itertools.combinations(favorite_bets, r)
        for combo in combinations:
            # Ensure all events are unique
            event_names = {bet['event'] for bet in combo}
            if len(event_names) < len(combo):
                continue  # Skip combinations with duplicate events

            # Calculate combined decimal odds and adjusted implied probability
            combined_decimal_odds = math.prod(bet['decimal_odds'] for bet in combo)
            combined_prob = math.prod(bet['adjusted_prob'] for bet in combo)
            min_discrepancy = min(bet['discrepancy'] for bet in combo)
            avg_rating = sum(bet['rating'] for bet in combo) / len(combo)
            # Convert combined decimal odds to American odds
            combined_american_odds = decimal_to_american(combined_decimal_odds)
            # Adjust combined odds range
            if odds_lower <= combined_american_odds <= odds_upper:
                # Calculate expected value
                payout = combined_decimal_odds - 1  # Profit per $1 bet
                expected_value = (combined_prob * payout) - ((1 - combined_prob) * 1)
                parlays.append({
                    'parlay': combo,
                    'combined_odds': combined_american_odds,
                    'implied_probability': combined_prob,
                    'expected_value': expected_value,
                    'min_discrepancy': min_discrepancy,
                    'avg_rating': avg_rating
                })
    return parlays

def main():
    # Interactive inputs
    num_parlays_to_output = int(input("Enter the number of top parlays to output: "))
    odds_lower = float(input("Enter the minimum combined American odds (e.g., 100 for +100): "))
    odds_upper = float(input("Enter the maximum combined American odds (e.g., 300 for +300): "))
    min_parlay_size = int(input("Enter the minimum parlay size (number of bets in a parlay): "))
    max_parlay_size = int(input("Enter the maximum parlay size (number of bets in a parlay): "))
    rating_threshold = float(input("Enter the minimum sport rating to consider (e.g., 3.5): "))

    # Ratings dictionary based on historical betting accuracy
    ratings = {
        'americanfootball_nfl': 4.0,
        'americanfootball_ncaaf': 3.5,
        'americanfootball_cfl': 2.5,
        'basketball_nba': 4.5,
        'basketball_ncaab': 3.5,
        'boxing_boxing': 5.0,
        'cricket_ipl': 4.0,
        'cricket_test_match': 3.5,
        'soccer_epl': 3.0,
        'soccer_uefa_champs_league': 3.0,
        'soccer_argentina_primera_division': 3.0,
        'icehockey_nhl': 2.5,
        'tennis_atp_us_open': 3.5,
        'mma_mixed_martial_arts': 5.0,
        # Add other sports with ratings as needed
    }

    # Read the CSV file
    df = pd.read_csv('Next30_Odds.csv')

    # Add a rating column to the DataFrame
    df['rating'] = df['sport'].map(ratings)

    # Drop rows with no rating or invalid data
    df = df.dropna(subset=['rating', 'odds1', 'odds2', 'team1', 'team2'])

    # Filter for sports with ratings above the threshold
    df = df[df['rating'] >= rating_threshold]

    # List to hold bets on favorites with adjusted odds range
    favorite_bets = []

    # Process each row to identify favorites
    for index, row in df.iterrows():
        try:
            odds1 = float(row['odds1'])
            odds2 = float(row['odds2'])
            team1 = row['team1']
            team2 = row['team2']
            event = f"{row['sport']} {row['event_time']}"
            rating = row['rating']
        except (ValueError, KeyError, TypeError):
            continue  # Skip rows with invalid data

        # Identify favorite and underdog
        if odds1 < odds2:
            favorite_odds = odds1
            underdog_odds = odds2
            favorite_team = team1
            underdog_team = team2
        else:
            favorite_odds = odds2
            underdog_odds = odds1
            favorite_team = team2
            underdog_team = team1

        # Adjusted odds range (-2000 to -500)
        if favorite_odds < 0 and -2000 <= favorite_odds <= -500:
            # Calculate implied probabilities
            favorite_implied_prob = american_to_implied_prob(favorite_odds)
            underdog_implied_prob = american_to_implied_prob(underdog_odds)
            # Adjust probabilities using sport rating
            adjusted_favorite_prob = adjust_implied_prob(favorite_implied_prob, rating, is_favorite=True)
            adjusted_underdog_prob = adjust_implied_prob(underdog_implied_prob, rating, is_favorite=False)
            # Ensure probabilities are between 0 and 1
            adjusted_favorite_prob = min(max(adjusted_favorite_prob, 0), 1)
            adjusted_underdog_prob = min(max(adjusted_underdog_prob, 0), 1)
            # Calculate discrepancy as ratio
            if adjusted_underdog_prob > 0:
                discrepancy = adjusted_favorite_prob / adjusted_underdog_prob
            else:
                discrepancy = float('inf')
            # Store the bet with discrepancy
            favorite_bets.append({
                'event': event,
                'team': favorite_team,
                'odds': favorite_odds,
                'decimal_odds': american_to_decimal(favorite_odds),
                'implied_prob': favorite_implied_prob,
                'adjusted_prob': adjusted_favorite_prob,
                'discrepancy': discrepancy,
                'rating': rating
            })

    # Check the number of favorite bets
    print(f"Number of favorite bets: {len(favorite_bets)}")

    # Sort favorite bets by adjusted probability and discrepancy (descending)
    favorite_bets = sorted(favorite_bets, key=lambda x: (-x['adjusted_prob'], -x['discrepancy']))

    # Generate parlays
    parlays = generate_parlays(
        favorite_bets,
        min_parlay_size,
        max_parlay_size,
        odds_lower,
        odds_upper
    )

    # Sort parlays from most likely to least likely based on implied probability and average rating
    sorted_parlays = sorted(parlays, key=lambda x: (-x['implied_probability'], -x['avg_rating']))

    # Output results
    total_parlays_generated = len(sorted_parlays)
    print(f"Generated {total_parlays_generated} parlays.")
    parlays_to_output = min(num_parlays_to_output, total_parlays_generated)
    print(f"Displaying top {parlays_to_output} parlays:\n")

    # Prepare data for CSV output
    parlay_output = []

    for idx, parlay in enumerate(sorted_parlays[:parlays_to_output], start=1):
        print(f"Parlay {idx}:")
        parlay_bets = []
        for bet in parlay['parlay']:
            print(f"  Event: {bet['event']}, Team: {bet['team']}, Odds: {bet['odds']}")
            parlay_bets.append(f"{bet['event']} - {bet['team']} ({bet['odds']})")
        print(f"  Combined Odds (American): +{parlay['combined_odds']:.2f}")
        print(f"  Implied Probability: {parlay['implied_probability'] * 100:.2f}%")
        print(f"  Average Rating: {parlay['avg_rating']:.2f}")
        print(f"  Expected Value per $1 bet: ${parlay['expected_value']:.2f}")
        print("-----")

        # Add to parlay_output for CSV
        parlay_output.append({
            'Parlay_Number': idx,
            'Parlay_Bets': ' | '.join(parlay_bets),
            'Combined_Odds': parlay['combined_odds'],
            'Implied_Probability': parlay['implied_probability'],
            'Expected_Value': parlay['expected_value'],
            'Average_Rating': parlay['avg_rating']
        })

    # Save parlays to CSV
    output_df = pd.DataFrame(parlay_output)
    output_df.to_csv('parlay_results.csv', index=False)
    print("Parlay results have been written to 'parlay_results.csv'.")

if __name__ == "__main__":
    main()


Number of favorite bets: 26
Generated 403051 parlays.
Displaying top 20 parlays:

Parlay 1:
  Event: americanfootball_ncaaf 2024-11-17 03:30:00, Team: UNLV Rebels, Odds: -1600.0
  Event: boxing_boxing 2024-11-16 16:00:00, Team: Oscar Collazo, Odds: -1100.0
  Event: americanfootball_nfl 2024-11-17 18:00:00, Team: Detroit Lions, Odds: -800.0
  Event: boxing_boxing 2024-11-16 00:00:00, Team: Lucas Bahdi, Odds: -800.0
  Event: soccer_uefa_champs_league 2024-11-26 20:00:00, Team: Manchester City, Odds: -550.0
  Event: americanfootball_ncaaf 2024-11-16 17:00:00, Team: Liberty Flames, Odds: -650.0
  Combined Odds (American): +100.04
  Implied Probability: 44.88%
  Average Rating: 4.22
  Expected Value per $1 bet: $-0.10
-----
Parlay 2:
  Event: americanfootball_ncaaf 2024-11-17 03:30:00, Team: UNLV Rebels, Odds: -1600.0
  Event: boxing_boxing 2024-11-16 16:00:00, Team: Oscar Collazo, Odds: -1100.0
  Event: americanfootball_nfl 2024-11-17 18:00:00, Team: Detroit Lions, Odds: -800.0
  Event: bo